In [ ]:
from generic_model import Model
from data_utils.keras_generator import Generator
from keras import optimizers
import keras.backend as KB
import keras.losses as Loss
import tensorflow as tf
from Preprocessing.Preprocessor import preprocessor 
from data_utils.Loss import custom_loss

In [ ]:
encoder_dir='E:/'
data_dir='D:/Data'
p_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':10,
        'format_':'h5',
        'ex_batch_size':1000,
        'file_examples':5000,
        'Preprocessor':preprocessor(encoder_dir),
        'static_weightage':False}

G=Generator(config=p_config)


In [ ]:
nn_config={'layer_shapes':[200,200,200,50,10],'Activation':'relu','Output':6,'Input_shape':477, 'model_type':'fc'}
my_model=Model(nn_config) 
my_model.model.compile(loss=custom_loss, optimizer=optimizers.Adam(lr=0.05),metrics=["acc"])





In [ ]:
my_model.model.fit_generator(generator= G, 
                    steps_per_epoch  = len(G), 
                    epochs           = 10, 
                    verbose          = 1,
#                     validation_data  = valid_batch,
#                     validation_steps = len(valid_batch),
                    max_queue_size   = 3)